In this notebook:
    - pull USGS data for length that each gage is in operation
    - potentially cut time scale of data based on when the flow has never been above a certain threshold
    - each of the target gages should have a dataframe (list of dataframes)

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import os
import requests
import time

In [2]:
import pickle
path="C:\Springboard\Github\gauge_info"
os.chdir(path)

In [3]:
# load the dataframe of targets and their corresponding features
dt = pickle.load(open("USGS_targets.pkl", "rb"))
dt

,USGS,lat,long,alt,basin,lng,feat_USGS,feat_NOAA
0,10140700,41.231819,111.984497,4285.00,Lower Weber,-111.984497,"[10126000, 10092700, 10010000, 10171000, 10105...","[BCNU1, BIUI1, GSPU1, JRSU1, PRZU1, LCJU1, LGN..."
1,10149000,40.118012,111.314622,6320.00,Spanish Fork,-111.314622,"[10164500, 10133800, 10133650, 10133600, 09313...","[AFPU1, ECAU1, ECPU1, MCLU1, PRHU1, PVHU1, SCJ..."
2,10155200,40.554398,111.433243,5691.59,Provo,-111.433243,"[10164500, 10131000, 10134500, 10132500, 10155...","[AFPU1, CIVU1, ECCU1, CRDU1, PVHU1, RBCU1, CLL..."
79,09064600,39.553875,106.402529,8078.37,Eagle,-106.402529,"[09057500, 09034250, 09034500, 09065100, 09063...","[BGMC2, CAWC2, HTSC2, CSSC2, RERC2, FRGC2, FPT..."
84,09081000,39.373317,107.083937,6470.00,Roaring Fork,-107.083937,"[09132095, 09070500, 09085100, 09081600, 09070...","[ACSC2, EGLC2, GCOC2, RCYC2, GPSC2, ENMC2, GEP..."
85,09073300,39.141100,106.774204,8120.00,Roaring Fork,-106.774204,"[09065100, 09112500, 09078600, 09080400, 09066...","[CSSC2, ALEC2, FPTC2, RUDC2, GRVC2, GUSC2, HUN..."
86,09075400,39.193833,106.833667,7882.00,Roaring Fork,-106.833667,"[09065100, 09112500, 09078600, 09080400, 09066...","[CSSC2, ALEC2, FPTC2, RUDC2, GRVC2, GUSC2, OHO..."
87,09033300,40.006892,105.848272,8274.00,Colorado Headwaters,-105.848272,"[09050700, 09010500, 09034250, 09019500, 09034...","[BLRC2, BAKC2, CAWC2, CBGC2, HTSC2, FRGC2, FRW..."
88,09359010,37.802774,107.672839,9245.98,Animas,-107.672839,"[09126000, 09118450, 09165000, 09074000, 09365...","[CMRC2, CRCC2, DRRC2, HUNC2, LPHC2, LFBC2, LFG..."
92,09358000,37.811108,107.659228,9290.00,Animas,-107.659228,"[09126000, 09118450, 09165000, 09074000, 09365...","[CMRC2, CRCC2, DRRC2, HUNC2, LPHC2, LFBC2, LFG..."


That looks good. Now what? :) <br>

How do I structure the data that I pull for each of these gages? <br>

I think we may need to pull these files individually and save them to a CSV. Then for the modeling, we load the CSV's that we need. Otherwise, it's too much memory.

In [ ]:
# do the same data pulling for the target USGS gages
for i, row in dt:
    if gage not in already_pulled:
        

In [ ]:
# list of gages that we already pulled data for
# for i, row in dt:
    # get list of USGS gages
    
    # for gage in USGS_list:
        # if gage is not in list of already pulled data
            # get CSV file of daily summary of water flow
            # save CSV file
            # add gage to list of already pulled data
            
        

In the notebook after that:
    - build models (using Lasso Regression, we believe)
    - set threshold for accuracy in predicting flow
    - throw out models that do not meet that accuracy
    - throw out features in models that are no longer needed
    - do other prep for putting models into production (test in real time and have accuracy feedback)